# BIDS Steering Group Election 2021

This file, along with `votes.csv`, reproduces the ballot counting procedure for the BIDS Steering Group election of 2021.

Voting was done according to a ranked choice voting scheme, and tallied using an Instant Runoff procedure.

In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from pprint import pprint

In [2]:
raw_votes = pd.read_csv('votes.csv', na_values='N/A', index_col="Timestamp", parse_dates=True)

In [3]:
def extract_name(field):
    return field.split("[")[1][:-1]

votes = raw_votes.rename(columns=extract_name)

Ballots are a numeric ranking, with the top choice (most preferable) of 1. N/A votes were permitted and are ensure a ballot will never count for a candidate.

In [4]:
votes[:3]

,Matthew Cieslak,Eric Earl,Helena Ledmyr,Robert Luke,Franco Pestilli,Petra Ritter,Ariel Rokem,Hao-Ting Wang
Timestamp,,,,,,,,
2021-10-26 18:56:53,1.0,2.0,3.0,4.0,5,6,7,8.0
2021-10-26 19:02:24,1.0,7.0,3.0,NaN,8,5,2,4.0
2021-10-26 19:14:23,8.0,7.0,4.0,1.0,2,3,5,6.0


In [5]:
def ranked_choices(vote):
    choices = votes.columns[~np.isnan(vote)]
    ranks = vote[~np.isnan(vote)]
    # Validate that no ballot had ties (should have been prevented by the form)
    assert len(ranks) == len(set(ranks))
    return list(choices[np.argsort(ranks)])

# Validate that ordering strategy works with NaNs
for vote in votes.values.astype(float)[:3]:
    print(vote)
    print(ranked_choices(vote))

[1. 2. 3. 4. 5. 6. 7. 8.]
['Matthew Cieslak', 'Eric Earl', 'Helena Ledmyr', 'Robert Luke', 'Franco Pestilli', 'Petra Ritter', 'Ariel Rokem', 'Hao-Ting Wang']
[ 1.  7.  3. nan  8.  5.  2.  4.]
['Matthew Cieslak', 'Ariel Rokem', 'Helena Ledmyr', 'Hao-Ting Wang', 'Petra Ritter', 'Eric Earl', 'Franco Pestilli']
[8. 7. 4. 1. 2. 3. 5. 6.]
['Robert Luke', 'Franco Pestilli', 'Petra Ritter', 'Helena Ledmyr', 'Ariel Rokem', 'Hao-Ting Wang', 'Eric Earl', 'Matthew Cieslak']


In [6]:
ballots = [ranked_choices(vote) for vote in votes.values.astype(float)]

Here we implement the instant runoff procedure.

In [7]:
def count_votes(ballots):
    # Find first choices, dropping empty ballots (can happen after first prune)
    first_choice = [ballot[0] for ballot in ballots if ballot]
    # Sort first choices by number of votes, most votes first
    return sorted(
        Counter(first_choice).items(),
        key=lambda x: x[1],
        reverse=True)

def prune(ballots, remove_candidate):
    # Filter the removed candidate from all ballots
    return [[cand for cand in ballot if cand != remove_candidate]
            for ballot in ballots]

def instant_runoff(ballots, round=1):
    tally = count_votes(ballots)

    print(f"Round {round}:")
    pprint(tally)
    
    if len(tally) <= 2:
        candidate, votes = tally.pop(0)
        print(f"--------")
        print(f"Winner: {candidate}, with {votes} votes")
    else:
        candidate, votes = tally.pop()
        print(f"Removing {candidate}, reallocating {votes} votes")
        print(f"--------")
        # Rerun with pruned ballots
        instant_runoff(prune(ballots, candidate), round=round+1)

In [8]:
instant_runoff(ballots)

Round 1:
[('Matthew Cieslak', 36),
 ('Ariel Rokem', 29),
 ('Robert Luke', 18),
 ('Petra Ritter', 14),
 ('Franco Pestilli', 12),
 ('Eric Earl', 11),
 ('Helena Ledmyr', 8),
 ('Hao-Ting Wang', 7)]
Removing Hao-Ting Wang, reallocating 7 votes
--------
Round 2:
[('Matthew Cieslak', 37),
 ('Ariel Rokem', 32),
 ('Robert Luke', 19),
 ('Petra Ritter', 14),
 ('Franco Pestilli', 13),
 ('Eric Earl', 12),
 ('Helena Ledmyr', 8)]
Removing Helena Ledmyr, reallocating 8 votes
--------
Round 3:
[('Matthew Cieslak', 37),
 ('Ariel Rokem', 34),
 ('Robert Luke', 19),
 ('Petra Ritter', 18),
 ('Eric Earl', 14),
 ('Franco Pestilli', 13)]
Removing Franco Pestilli, reallocating 13 votes
--------
Round 4:
[('Matthew Cieslak', 38),
 ('Ariel Rokem', 36),
 ('Petra Ritter', 24),
 ('Robert Luke', 21),
 ('Eric Earl', 16)]
Removing Eric Earl, reallocating 16 votes
--------
Round 5:
[('Ariel Rokem', 43),
 ('Matthew Cieslak', 39),
 ('Petra Ritter', 30),
 ('Robert Luke', 23)]
Removing Robert Luke, reallocating 23 votes
---